In [6]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Here we set the model. Feel free to replace it with any Hugging Face chat/instruct model (just check to use appropriate "roles" as defined below).

In [4]:
models_dict = {
    "Zephyr 7B Alpha": "HuggingFaceH4/zephyr-7b-alpha", 
    "Zephyr 7B Beta": "HuggingFaceH4/zephyr-7b-beta", 
}

model_id = models_dict["Zephyr 7B Beta"]

In [7]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

(…)beta/resolve/main/generation_config.json: 100%|██████████| 111/111 [00:00<00:00, 694kB/s]
(…)-beta/resolve/main/tokenizer_config.json: 100%|██████████| 1.43k/1.43k [00:00<00:00, 9.26MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 330MB/s]
(…)phyr-7b-beta/resolve/main/tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 26.8MB/s]
(…)r-7b-beta/resolve/main/added_tokens.json: 100%|██████████| 42.0/42.0 [00:00<00:00, 254kB/s]
(…)eta/resolve/main/special_tokens_map.json: 100%|██████████| 168/168 [00:00<00:00, 938kB/s]


Now define the role you'd like the chatbot to assume.

In [8]:
job_description = "You are a Product Photoshoot expert. Please answer my questions."

Below we run the model! Use "stop" or "exit" to exit question answering generation. You can ignore the `UserWarning: You have modified the pretrained model configuration...` warning.

In [ ]:
messages = [
    {
        "role": "system",
        "content": job_description,
    },
]
    
exit_terms = ["stop", "exit"]

while True:
    question = input("\nQuestion: ")
    if question.lower() in exit_terms: 
        break
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    response_start = output.rfind('<|assistant|>')
    print(output[response_start + len('<|assistant|>'):])